# 用LDA去找出特徵值

In [1]:
import pandas as pd
#把語料讀出來
df_corpus = pd.read_csv('./ppt_clean_cut_articles.csv', encoding='utf-8')
len(df_corpus)

94781

In [2]:
df_corpus = df_corpus.dropna()
df_corpus = df_corpus.reset_index(drop=True)
df_corpus.head(10)

,article,time
0,好 FITNESS 知道 方向 似乎 不太 明確 FITNESS 版主 範圍 不方便 干涉 ...,2005-05-10 00:00:00
1,貴版 準備 導向 方向 有氧運動 不就 FITNESS 體適能 範圍 有氧運動 廣就 很像 ...,2005-05-10 00:00:00
2,有氧 好棒 一起 跳 有氧 蹦蹦跳跳 好 健康,2005-05-10 00:00:00
3,有氧運動 愛好 希望 能夠 認識 跳 有氧 同好 祝版 運 昌隆 知道 靠近 一點,2005-05-10 00:00:00
4,恭喜 開板 最近 也想 跳 有氧 中正 萬華區 跳 有氧 好地方 謝謝,2005-05-10 00:00:00
5,恭喜 恭喜 以後 一定 常來 逛逛 好久 終於 成立 感動,2005-05-10 00:00:00
6,恭喜 很喜歡 跳 有氧 算是 逃家 孩子 正在 人 轉讓 有氧 上課,2005-05-10 00:00:00
7,台南 唸書 學期 會在 顧曉蘭 跳 有氧 放假 回 桃園 不可能 跳 問問 桃園 哪裡 有氧...,2005-05-10 00:00:00
8,恭喜 開版 喜愛 有氧 一枚 報到 加入 最愛,2005-05-10 00:00:00
9,恭喜 開版 意外 發現 真是 開心 熱愛 有氧 一定會 逛逛 要不要 來個 版員 名單 交流...,2005-05-10 00:00:00


In [3]:
df_idf = df_corpus.loc[0:, 'article']
print(df_idf.head(10))

0    好 FITNESS 知道 方向 似乎 不太 明確 FITNESS 版主 範圍 不方便 干涉 ...
1    貴版 準備 導向 方向 有氧運動 不就 FITNESS 體適能 範圍 有氧運動 廣就 很像 ...
2                              有氧 好棒 一起 跳 有氧 蹦蹦跳跳 好 健康
3            有氧運動 愛好 希望 能夠 認識 跳 有氧 同好 祝版 運 昌隆 知道 靠近 一點
4                  恭喜 開板 最近 也想 跳 有氧 中正 萬華區 跳 有氧 好地方 謝謝
5                        恭喜 恭喜 以後 一定 常來 逛逛 好久 終於 成立 感動
6                   恭喜 很喜歡 跳 有氧 算是 逃家 孩子 正在 人 轉讓 有氧 上課
7    台南 唸書 學期 會在 顧曉蘭 跳 有氧 放假 回 桃園 不可能 跳 問問 桃園 哪裡 有氧...
8                              恭喜 開版 喜愛 有氧 一枚 報到 加入 最愛
9    恭喜 開版 意外 發現 真是 開心 熱愛 有氧 一定會 逛逛 要不要 來個 版員 名單 交流...
Name: article, dtype: object


In [4]:
#一次可以取代多類的字
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [6]:
#把所有世界健身房同同意字都轉成相同的字
gym_dict_1 = {'WORLDGYM':'世界健身', 'WG':'世界健身', '世界健身房':'世界健身', '世界健身中心':'世界健身', '萵苣':'世界健身',
           '極限健身中心':'極限', 'EXTREME':'極限', '極限健身房':'極限', '極限運動中心': '極限',
           '健工':'健身工廠', 'FITNESSFACTORY':'健身工廠',
           'BEING':'統一', '統一健身房':'統一', '統一健身俱樂部':'統一', '統一健身':'統一' ,
           'TRUEFITNESS':'全真', 'TRUEYOGA':'全真', '全真健身':'全真', '全真瑜珈健身中心':'全真' }
for k in range (0, len(df_idf)):
    df_idf[k] = replace_all(df_idf[k], gym_dict_1)


gym_dict_2 = {'全真':'全真健身','統一':'統一健身','FACTORY':'健身工廠','極限':'極限健身'}
for k in range(0, len(df_idf)):
    df_idf[k] = replace_all(df_idf[k], gym_dict_2)

In [74]:
#把各健身房的資料取出後再重新設過index
WGs = df_idf[df_idf.str.contains("世界健身", na=False)].reset_index(drop =True)
GFs = df_idf[df_idf.str.contains("健身工廠", na=False)].reset_index(drop =True)
EXs = df_idf[df_idf.str.contains("極限健身", na=False)].reset_index(drop =True)
TFs = df_idf[df_idf.str.contains("全真健身", na=False)].reset_index(drop =True)
BEs = df_idf[df_idf.str.contains("統一健身", na=False)].reset_index(drop =True)
BEs.head(10)

0    推荐 老師 JAMEY 所在 地區 台北 BURNFUN 地名店 名 時間 星期三 晚上 星...
1    減肥 外食 人 真的 可憐 包括 在內 外面 食物 口味 油 重 身材 殺手 卡路里 難算 ...
2    便秘 燕麥片 超 有用 弄 美祿 三合一 麥片 阿華田 芝麻糊 反正 沖泡 類的 東西 加 ...
3    以前 爬文 看過 用過 這幾周 有心得 分享 心得 那種 實際 體重 輕五 公斤 人 臉 比...
4    分享 秘方 都有 試過 腸子 很難 伺候 每天 同樣 方法 沒效 最近 發現 完全 不用 花...
5    最近 改吃 半素 外加 調整 生活 作息 消化系統 有幫助 生活習慣 改成 吃飯 挑素 吃 ...
6    推 FARRO 補微 試試 FARRO 目前 吃 晚餐 省掉 中餐 FARRO 天瘦 停滯期...
7    這兩天 跑過 體驗 並不是 心得 希望 能夠 分享 人想 星期六 這兩天 工作 之餘 健身房...
8    七月 學弟 笑說 明顯 變胖 知道 最 胖胖 六月 畢業 送舊 一堆 吃 一堆 燒烤 謝師宴...
9    一點點 看不懂 提出 問題 聚 要有 交流 心得 感情 奔放 跳 有氧 情況 時間 不一樣 ...
Name: article, dtype: object

In [75]:
#把各健身房的CORPUS變成可以丟入LDA的樣子
WG_corpus = []
for k in range(0, len(WGs)):
    WG_corpus.append(WGs[k].split(" "))
GF_corpus = []
for k in range(0, len(GFs)):
    GF_corpus.append(GFs[k].split(" "))
EX_corpus = []
for k in range(0, len(EXs)):
    EX_corpus.append(EXs[k].split(" "))
TF_corpus = []
for k in range(0, len(TFs)):
    TF_corpus.append(TFs[k].split(" "))
BE_corpus = []
for k in range(0, len(BEs)):
    BE_corpus.append(BEs[k].split(" "))
WG_corpus

[['世界健身',
  '第一次',
  '有氧課程',
  '分別',
  'BODYPUMP',
  '槓鈴',
  '有氧',
  'BODYCOMBAT',
  '拳擊',
  '有氧',
  '全身',
  '痠痛',
  '中',
  '台中',
  'SOGO',
  '二館',
  '世界健身',
  '幫',
  '解說',
  'KENNYBODYPUMP',
  '進入',
  '狀況',
  '下次',
  '上完',
  '分享',
  'BODYCOMBAT',
  '超有',
  'POWER',
  '實在',
  '享受',
  '搏擊',
  '大喊',
  '喝哈',
  '已經',
  '很累',
  '連上',
  '兩堂',
  '好',
  '疲累',
  '卻要',
  '硬撐',
  '下去',
  '整個',
  '發洩',
  '喝',
  '哈中',
  '上完',
  'PUMP',
  '大腿',
  '肌',
  '超痠',
  '害',
  'COMBAT',
  '熱身',
  '拉筋',
  '發抖',
  'ORZ',
  '拳擊',
  '有氧',
  '很棒',
  '女生',
  '綁起',
  '馬尾',
  '時邊',
  '跳邊',
  '甩',
  '馬尾',
  '運動',
  '老師',
  '兩邊',
  '作對',
  '動作',
  '算',
  '互動',
  '盡情',
  '飆汗',
  '運動',
  '爽',
  '明天',
  '一定會',
  '完全',
  '無力',
  'ORZ',
  '再來',
  '還想',
  '課程',
  '階梯',
  '有氧',
  'BODYJAM',
  '拉丁',
  '有氧',
  '瞄',
  '飛輪',
  '教室',
  '超猛',
  '爬坡',
  '下坡',
  '情況',
  '模擬',
  '卯足',
  '全勁',
  '飆車',
  '雕塑',
  '下半身',
  '有效',
  '下回',
  '挑戰',
  '看看',
  '體能',
  '運動',
  '真的',
  '很棒',
  '一起',
  '享受',
  '有氧',
  '魅力',
  '貓咪',
  '好朋友'],
 ['拳

# 世界健身 LDA

In [110]:
from gensim import corpora
from collections import defaultdict

#去除在所有文章中只出現一次的詞
#frequency[token]會回這個字出現幾次
frequency = defaultdict(int)
for text in WG_corpus:
    for token in text:
        frequency[token] += 1
        
WG_corpus = [[token for token in text if frequency[token] > 1]
         for text in WG_corpus]


# Creating term dictionary of corpus, where each unique term is assigned an index. 
dictionary_WG = corpora.Dictionary(WG_corpus)
dictionary_WG.filter_extremes(no_below=4, no_above=0.3)

# 在執行完前面的過濾操作以後，可能會造成單詞的序號之間有空隙，這時就可以使用該函數來對詞典來進行重新排序，去掉這些空隙。
dictionary_WG.compactify()
doc_term_matrix_WG = [dictionary_WG.doc2bow(x) for x in WG_corpus]

In [111]:
from gensim import corpora, models, similarities
#models可以產生tfidf-model, lsi-model, lda-model

# 注意，目前只是生成了一個模型，但這是類似於生成器，並不是將對應的corpus轉化後的結果。
tfidf_model_WG = models.TfidfModel(doc_term_matrix_WG) 
#對tf-idf模型而言，裡面存儲有各個單詞的
# 詞頻，文頻等信息。想要將文檔轉化成tf-idf模式表示的向量，還要使用如下命令
corpus_tfidf_WG = tfidf_model[doc_term_matrix_WG]

In [112]:
#建立LDA MODEL
from gensim.models.ldamodel import LdaModel as Lda
WG_lda = Lda(doc_term_matrix_WG, num_topics=8, id2word = dictionary_WG, passes=50, iterations=500)

In [113]:
#print out all topics
WG_lda.print_topics(num_topics=10, num_words=50)

[(0,
  '0.014*"脂肪" + 0.010*"蛋白質" + 0.010*"乳清" + 0.010*"飲食" + 0.009*"熱量" + 0.007*"身體" + 0.007*"中" + 0.007*"研究" + 0.007*"食物" + 0.006*"能量" + 0.006*"吃" + 0.006*"消耗" + 0.006*"人" + 0.006*"攝取" + 0.005*"進行" + 0.005*"碳水化合物" + 0.005*"蛋白" + 0.005*"增加" + 0.004*"高" + 0.004*"選擇" + 0.004*"減肥" + 0.004*"健康" + 0.004*"低" + 0.004*"營養" + 0.004*"減脂" + 0.004*"補充" + 0.004*"胸部" + 0.003*"口味" + 0.003*"吸收" + 0.003*"產生" + 0.003*"效果" + 0.003*"燃燒" + 0.003*"巧克力" + 0.003*"肌肉" + 0.003*"減少" + 0.003*"磅" + 0.003*"造成" + 0.003*"最佳" + 0.003*"人體" + 0.003*"更多" + 0.003*"降低" + 0.003*"減重" + 0.003*"代謝" + 0.003*"使用" + 0.003*"快速" + 0.003*"消化" + 0.003*"高蛋白" + 0.003*"需要" + 0.003*"維持" + 0.002*"系統"'),
 (1,
  '0.029*"吃" + 0.014*"體重" + 0.008*"時間" + 0.008*"減肥" + 0.006*"熱量" + 0.006*"飲食" + 0.006*"卡" + 0.006*"晚餐" + 0.006*"早餐" + 0.006*"重訓" + 0.005*"分鐘" + 0.005*"脂肪" + 0.005*"公斤" + 0.005*"每天" + 0.005*"喝" + 0.005*"左右" + 0.005*"體脂" + 0.004*"瘦" + 0.004*"午餐" + 0.004*"問題" + 0.004*"習慣" + 0.004*"水果" + 0.004*"體脂率" + 0.003*"建議" + 0.003*"身高" + 0.003*"青菜" 

In [55]:
corpuslda = WG_lda[corpus_tfidf]
list(corpuslda)

[[(0, 0.44246688),
  (3, 0.037083734),
  (4, 0.13566771),
  (7, 0.19348744),
  (8, 0.017723737),
  (9, 0.1369085)],
 [(0, 0.29979002),
  (1, 0.015185942),
  (2, 0.072234325),
  (3, 0.028682774),
  (4, 0.077539355),
  (5, 0.015194601),
  (6, 0.015198123),
  (7, 0.44578367),
  (8, 0.015188255),
  (9, 0.0152029265)],
 [(0, 0.1930488),
  (1, 0.20808402),
  (2, 0.27787077),
  (3, 0.01064376),
  (4, 0.015937688),
  (5, 0.0106427055),
  (6, 0.2518453),
  (7, 0.01063927),
  (8, 0.010648016),
  (9, 0.010639707)],
 [(0, 0.19457816),
  (1, 0.23182833),
  (2, 0.26204637),
  (3, 0.09428571),
  (4, 0.012989989),
  (6, 0.16759494)],
 [(0, 0.045668803),
  (1, 0.22198895),
  (2, 0.26229736),
  (3, 0.14239796),
  (6, 0.12231583),
  (7, 0.16857389),
  (8, 0.012055233)],
 [(0, 0.5219445),
  (1, 0.01948303),
  (2, 0.019484328),
  (3, 0.18074752),
  (4, 0.019483216),
  (5, 0.020185197),
  (6, 0.090782374),
  (7, 0.020170266),
  (8, 0.08802841),
  (9, 0.019691147)],
 [(0, 0.10031109),
  (1, 0.21500283),
  (2

In [114]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
WG = pyLDAvis.gensim.prepare(WG_lda, doc_term_matrix_WG, dictionary_WG)
pyLDAvis.save_html(WG, 'WG_LDA.html')

C:\Users\Java\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


# 健身工廠 LDA

In [105]:
from gensim import corpora
from collections import defaultdict

#去除在所有文章中只出現一次的詞
#frequency[token]會回這個字出現幾次
frequency = defaultdict(int)
for text in GF_corpus:
    for token in text:
        frequency[token] += 1
        
GF_corpus = [[token for token in text if frequency[token] > 1]
         for text in GF_corpus]

# Creating term dictionary of corpus, where each unique term is assigned an index. 
dictionary_GF = corpora.Dictionary(GF_corpus)
dictionary_GF.filter_extremes(no_below=4, no_above=0.3)

# 在執行完前面的過濾操作以後，可能會造成單詞的序號之間有空隙，這時就可以使用該函數來對詞典來進行重新排序，去掉這些空隙。
dictionary_GF.compactify()
doc_term_matrix_GF = [dictionary_GF.doc2bow(x) for x in GF_corpus]

In [106]:
from gensim import corpora, models, similarities
#models可以產生tfidf-model, lsi-model, lda-model

# 注意，目前只是生成了一個模型，但這是類似於生成器，並不是將對應的corpus轉化後的結果。
tfidf_model_GF = models.TfidfModel(doc_term_matrix_GF) 
#對tf-idf模型而言，裡面存儲有各個單詞的
# 詞頻，文頻等信息。想要將文檔轉化成tf-idf模式表示的向量，還要使用如下命令
corpus_tfidf_GF = tfidf_model[doc_term_matrix_GF]

In [107]:
#建立LDA MODEL
from gensim.models.ldamodel import LdaModel as Lda
GF_lda = Lda(doc_term_matrix_GF, num_topics=8, id2word = dictionary_GF, passes=50, iterations=500)

In [108]:
#print out all topics
GF_lda.print_topics(num_topics=8, num_words=50)

[(0,
  '0.016*"吃" + 0.010*"飲食" + 0.009*"體重" + 0.008*"重訓" + 0.008*"時間" + 0.007*"身體" + 0.006*"有氧" + 0.006*"脂肪" + 0.006*"喝" + 0.006*"目前" + 0.005*"減肥" + 0.005*"體脂" + 0.005*"做" + 0.005*"問題" + 0.004*"訓練" + 0.004*"分鐘" + 0.004*"習慣" + 0.004*"人" + 0.004*"左右" + 0.004*"肌肉" + 0.004*"瘦" + 0.004*"健康" + 0.004*"熱量" + 0.004*"每天" + 0.004*"建議" + 0.004*"方式" + 0.004*"乳清" + 0.004*"好" + 0.003*"公斤" + 0.003*"大卡" + 0.003*"晚餐" + 0.003*"小時" + 0.003*"持續" + 0.003*"蛋白質" + 0.003*"之後" + 0.003*"身高" + 0.003*"喝水" + 0.003*"否" + 0.003*"比較" + 0.003*"營養" + 0.003*"健身" + 0.003*"活動" + 0.003*"作息" + 0.003*"狀況" + 0.003*"買" + 0.002*"早餐" + 0.002*"工作" + 0.002*"胖" + 0.002*"調整" + 0.002*"已經"'),
 (1,
  '0.037*"YES" + 0.023*"新手" + 0.018*"晚上" + 0.018*"練" + 0.013*"台北市" + 0.012*"下午" + 0.011*"平日" + 0.011*"一起" + 0.010*"台南" + 0.008*"不一定" + 0.008*"重訓" + 0.008*"早上" + 0.008*"世界" + 0.007*"台大" + 0.007*"新體" + 0.007*"中心" + 0.007*"統領" + 0.007*"六日" + 0.006*"台北" + 0.006*"西門" + 0.006*"規律" + 0.006*"找" + 0.005*"指導" + 0.005*"深呼吸" + 0.005*"身材" + 0.005*"體育館" + 

In [61]:
corpuslda = GF_lda[corpus_tfidf]
list(corpuslda)

[[(0, 0.50376976),
  (1, 0.011470568),
  (2, 0.011468849),
  (3, 0.011484465),
  (4, 0.34157363),
  (5, 0.020050643),
  (6, 0.08658607),
  (7, 0.013596038)],
 [(0, 0.28470647),
  (1, 0.019005995),
  (2, 0.019015133),
  (3, 0.40930843),
  (4, 0.12319878),
  (5, 0.018994458),
  (6, 0.10679012),
  (7, 0.018980594)],
 [(0, 0.50943285),
  (1, 0.013312896),
  (2, 0.013312329),
  (3, 0.10269373),
  (4, 0.2315894),
  (5, 0.10301103),
  (6, 0.013336201),
  (7, 0.013311529)],
 [(0, 0.41411728),
  (1, 0.011474091),
  (2, 0.01147235),
  (3, 0.049143005),
  (4, 0.37643576),
  (5, 0.114356704),
  (6, 0.0115092145),
  (7, 0.011491607)],
 [(0, 0.22147086),
  (1, 0.021854019),
  (2, 0.14979313),
  (3, 0.4325382),
  (4, 0.05466598),
  (5, 0.10002337),
  (6, 0.010053949)],
 [(0, 0.65082353),
  (1, 0.024373902),
  (2, 0.06803654),
  (3, 0.02449785),
  (4, 0.024487179),
  (5, 0.08704241),
  (6, 0.02438499),
  (7, 0.09635361)],
 [(0, 0.021259587),
  (1, 0.0131796375),
  (2, 0.37215102),
  (3, 0.1103513),
  

In [109]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
GF = pyLDAvis.gensim.prepare(GF_lda, doc_term_matrix_GF, dictionary_GF)
pyLDAvis.save_html(GF, 'GF_LDA.html')

C:\Users\Java\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


# 極限健身 LDA

In [99]:
from gensim import corpora
from collections import defaultdict

#去除在所有文章中只出現一次的詞
#frequency[token]會回這個字出現幾次
frequency = defaultdict(int)
for text in EX_corpus:
    for token in text:
        frequency[token] += 1
        
EX_corpus = [[token for token in text if frequency[token] > 1]
         for text in EX_corpus]

# Creating term dictionary of corpus, where each unique term is assigned an index. 
dictionary_EX = corpora.Dictionary(EX_corpus)
dictionary_EX.filter_extremes(no_below=4, no_above=0.3)

# 在執行完前面的過濾操作以後，可能會造成單詞的序號之間有空隙，這時就可以使用該函數來對詞典來進行重新排序，去掉這些空隙。
dictionary_EX.compactify()
doc_term_matrix_EX = [dictionary_EX.doc2bow(x) for x in EX_corpus]

In [100]:
from gensim import corpora, models, similarities
#models可以產生tfidf-model, lsi-model, lda-model

# 注意，目前只是生成了一個模型，但這是類似於生成器，並不是將對應的corpus轉化後的結果。
tfidf_model_EX = models.TfidfModel(doc_term_matrix_EX) 
#對tf-idf模型而言，裡面存儲有各個單詞的
# 詞頻，文頻等信息。想要將文檔轉化成tf-idf模式表示的向量，還要使用如下命令
corpus_tfidf_EX = tfidf_model[doc_term_matrix_EX]

In [101]:
#建立LDA MODEL
from gensim.models.ldamodel import LdaModel as Lda
EX_lda = Lda(doc_term_matrix_EX, num_topics=8, id2word = dictionary_EX, passes=50, iterations=500)

In [102]:
#print out all topics
EX_lda.print_topics(num_topics=10, num_words=50)

[(0,
  '0.028*"教練" + 0.017*"健身房" + 0.014*"課程" + 0.013*"課" + 0.012*"轉讓" + 0.011*"會員" + 0.011*"有氧" + 0.008*"器材" + 0.007*"使用" + 0.007*"會籍" + 0.007*"健身" + 0.006*"體驗" + 0.006*"橘子" + 0.006*"AEROBICS" + 0.006*"謝謝" + 0.006*"朋友" + 0.006*"站" + 0.006*"業務" + 0.005*"一起" + 0.005*"目前" + 0.005*"桃園" + 0.005*"歡迎" + 0.005*"老師" + 0.005*"買" + 0.005*"活潑" + 0.004*"方式" + 0.004*"加入" + 0.004*"世界健身" + 0.004*"體育場" + 0.004*"可愛" + 0.004*"聰明" + 0.004*"武術" + 0.004*"汗水" + 0.004*"人稱" + 0.004*"水上" + 0.004*"OKCOOLPTTSPORT" + 0.004*"冰上" + 0.004*"膽識" + 0.004*"清純" + 0.004*"天真無邪" + 0.004*"鬥志" + 0.004*"那顆" + 0.004*"希望" + 0.004*"價格" + 0.004*"推薦" + 0.004*"體操" + 0.004*"國家" + 0.004*"費用" + 0.003*"堂" + 0.003*"教室"'),
 (1,
  '0.016*"大卡" + 0.012*"吃" + 0.011*"熱量" + 0.010*"脂肪" + 0.009*"飲食" + 0.005*"攝取" + 0.005*"HIIT" + 0.005*"減肥" + 0.005*"建議" + 0.005*"營養" + 0.005*"份" + 0.005*"喝水" + 0.004*"燃燒" + 0.004*"體重" + 0.004*"水果" + 0.004*"分鐘" + 0.004*"基代" + 0.004*"計畫" + 0.004*"食物" + 0.004*"每日" + 0.004*"蛋白質" + 0.003*"代謝" + 0.003*"沒事" + 0.003*"青菜" + 

In [ ]:
corpuslda = EX_lda[corpus_tfidf]
list(corpuslda)

In [103]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
EX = pyLDAvis.gensim.prepare(EX_lda, doc_term_matrix_EX, dictionary_EX)
pyLDAvis.save_html(EX, 'EX_LDA.html')

C:\Users\Java\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


# 全真健身 LDA

In [94]:
from gensim import corpora
from collections import defaultdict

#去除在所有文章中只出現一次的詞
#frequency[token]會回這個字出現幾次
frequency = defaultdict(int)
for text in TF_corpus:
    for token in text:
        frequency[token] += 1
        
TF_corpus = [[token for token in text if frequency[token] > 1]
         for text in TF_corpus]

# Creating term dictionary of corpus, where each unique term is assigned an index. 
dictionary_TF = corpora.Dictionary(TF_corpus)
dictionary_TF.filter_extremes(no_below=4, no_above=0.3)

# 在執行完前面的過濾操作以後，可能會造成單詞的序號之間有空隙，這時就可以使用該函數來對詞典來進行重新排序，去掉這些空隙。
dictionary_TF.compactify()
doc_term_matrix_TF = [dictionary_TF.doc2bow(x) for x in TF_corpus]

In [95]:
from gensim import corpora, models, similarities
#models可以產生tfidf-model, lsi-model, lda-model

# 注意，目前只是生成了一個模型，但這是類似於生成器，並不是將對應的corpus轉化後的結果。
tfidf_model_TF = models.TfidfModel(doc_term_matrix_TF) 
#對tf-idf模型而言，裡面存儲有各個單詞的
# 詞頻，文頻等信息。想要將文檔轉化成tf-idf模式表示的向量，還要使用如下命令
corpus_tfidf_TF = tfidf_model[doc_term_matrix_TF]

In [96]:
#建立LDA MODEL
from gensim.models.ldamodel import LdaModel as Lda
TF_lda = Lda(doc_term_matrix_TF, num_topics=8, id2word = dictionary_TF, passes=50, iterations=500)

In [97]:
#print out all topics
TF_lda.print_topics(num_topics=8, num_words=50)

[(0,
  '0.019*"商品" + 0.015*"館" + 0.014*"會員" + 0.013*"體驗卡" + 0.012*"開卡" + 0.011*"面交" + 0.011*"備註" + 0.011*"內湖" + 0.010*"體驗" + 0.010*"數量" + 0.010*"徵求" + 0.010*"費用" + 0.009*"時間" + 0.009*"古亭" + 0.008*"需要" + 0.008*"需" + 0.008*"中壢" + 0.008*"TRUE" + 0.008*"一次" + 0.008*"無" + 0.008*"地點" + 0.008*"討論" + 0.007*"一堂" + 0.007*"台北" + 0.007*"出售" + 0.007*"狀況" + 0.006*"可使用" + 0.006*"環亞" + 0.006*"來信" + 0.006*"報價" + 0.006*"教練" + 0.006*"單點" + 0.006*"郵寄" + 0.006*"瑜珈" + 0.006*"團體" + 0.006*"堂" + 0.006*"參考" + 0.006*"全新" + 0.005*"年滿" + 0.005*"張" + 0.005*"免費" + 0.005*"歲" + 0.005*"場所" + 0.005*"新舊" + 0.005*"私人" + 0.005*"進出" + 0.005*"左右" + 0.005*"意者" + 0.005*"上課" + 0.005*"捷運"'),
 (1,
  '0.019*"運動" + 0.018*"一年" + 0.016*"健身房" + 0.013*"希望" + 0.012*"業務" + 0.011*"加入" + 0.010*"最近" + 0.010*"課" + 0.010*"古亭" + 0.010*"價錢" + 0.009*"教練" + 0.009*"一起" + 0.009*"器材" + 0.009*"健身" + 0.008*"會員" + 0.007*"左右" + 0.007*"台中" + 0.007*"目前" + 0.007*"徵求" + 0.007*"月費" + 0.007*"比較" + 0.006*"好" + 0.006*"有沒有" + 0.006*"人" + 0.006*"附近" + 0.006*"方案" 

In [ ]:
corpuslda = TF_lda[corpus_tfidf]
list(corpuslda)

In [98]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
TF = pyLDAvis.gensim.prepare(TF_lda, doc_term_matrix_TF, dictionary_TF)
pyLDAvis.save_html(TF, 'TF_LDA.html')

C:\Users\Java\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


# 統一健身 LDA

In [88]:
from gensim import corpora
from collections import defaultdict

#去除在所有文章中只出現一次的詞
#frequency[token]會回這個字出現幾次
frequency = defaultdict(int)
for text in BE_corpus:
    for token in text:
        frequency[token] += 1
        
BE_corpus = [[token for token in text if frequency[token] > 1]
         for text in BE_corpus]

# Creating term dictionary of corpus, where each unique term is assigned an index. 
dictionary_BE = corpora.Dictionary(BE_corpus)
dictionary_BE.filter_extremes(no_below=4, no_above=0.3)

# 在執行完前面的過濾操作以後，可能會造成單詞的序號之間有空隙，這時就可以使用該函數來對詞典來進行重新排序，去掉這些空隙。
dictionary_BE.compactify()
doc_term_matrix_BE = [dictionary_BE.doc2bow(x) for x in BE_corpus]

In [89]:
from gensim import corpora, models, similarities
#models可以產生tfidf-model, lsi-model, lda-model

# 注意，目前只是生成了一個模型，但這是類似於生成器，並不是將對應的corpus轉化後的結果。
tfidf_model_BE = models.TfidfModel(doc_term_matrix_BE) 
#對tf-idf模型而言，裡面存儲有各個單詞的
# 詞頻，文頻等信息。想要將文檔轉化成tf-idf模式表示的向量，還要使用如下命令
corpus_tfidf_BE = tfidf_model[doc_term_matrix_BE]

In [90]:
#建立LDA MODEL
from gensim.models.ldamodel import LdaModel as Lda
BE_lda = Lda(doc_term_matrix_BE, num_topics=8, id2word = dictionary_BE, passes=50, iterations=500)

In [91]:
#print out all topics
BE_lda.print_topics(num_topics=8, num_words=50)

[(0,
  '0.017*"課程" + 0.011*"會員" + 0.010*"教練" + 0.009*"報名" + 0.009*"有氧" + 0.008*"健身房" + 0.008*"伊士邦" + 0.008*"上課" + 0.007*"價格" + 0.007*"老師" + 0.007*"教室" + 0.006*"世界健身" + 0.006*"健身" + 0.006*"課" + 0.005*"優惠" + 0.005*"台北" + 0.005*"元" + 0.005*"會籍" + 0.005*"瑜珈" + 0.005*"業務" + 0.005*"使用" + 0.005*"費用" + 0.004*"俱樂部" + 0.004*"最" + 0.004*"公司" + 0.004*"電話" + 0.004*"轉讓" + 0.004*"銀行" + 0.004*"中心" + 0.004*"亞力山大" + 0.004*"健身工廠" + 0.003*"現場" + 0.003*"月費" + 0.003*"器材" + 0.003*"服務" + 0.003*"選擇" + 0.003*"方案" + 0.003*"一年" + 0.003*"找" + 0.003*"便宜" + 0.003*"申請" + 0.003*"專業" + 0.003*"方式" + 0.003*"直接" + 0.003*"購買" + 0.003*"站" + 0.003*"舞蹈" + 0.003*"通知" + 0.003*"世界" + 0.003*"頂級"'),
 (1,
  '0.012*"蛋白質" + 0.007*"食物" + 0.006*"營養" + 0.006*"攝取" + 0.006*"高" + 0.005*"低" + 0.005*"乳清" + 0.005*"蛋白" + 0.005*"中" + 0.005*"補充" + 0.005*"研究" + 0.005*"減重" + 0.004*"高蛋白" + 0.004*"高纖" + 0.004*"需要" + 0.004*"東西" + 0.004*"碳水化合物" + 0.003*"營養師" + 0.003*"效果" + 0.003*"能量" + 0.003*"選擇" + 0.003*"食品" + 0.003*"訓練" + 0.003*"纖維" + 0.003*"代謝" + 0

In [92]:
corpuslda = BE_lda[corpus_tfidf]
list(corpuslda)

[[(0, 0.18627806),
  (1, 0.011555389),
  (2, 0.5344514),
  (3, 0.19795702),
  (4, 0.011494869),
  (5, 0.011481162),
  (6, 0.011710727),
  (7, 0.03507132)],
 [(0, 0.09426118),
  (1, 0.019106083),
  (2, 0.6973449),
  (3, 0.01902048),
  (4, 0.019019958),
  (5, 0.11317007),
  (6, 0.019015428),
  (7, 0.01906186)],
 [(0, 0.17563151),
  (1, 0.013486476),
  (2, 0.67450196),
  (3, 0.070175625),
  (4, 0.013317555),
  (5, 0.013356454),
  (6, 0.01336288),
  (7, 0.026167594)],
 [(0, 0.1561661),
  (1, 0.011664015),
  (2, 0.7005322),
  (3, 0.0769534),
  (4, 0.011475519),
  (5, 0.011503743),
  (6, 0.011510251),
  (7, 0.020194773)],
 [(0, 0.015837396),
  (1, 0.23111302),
  (2, 0.55325663),
  (3, 0.010115075),
  (4, 0.01022086),
  (7, 0.16026114)],
 [(0, 0.19031549),
  (1, 0.024374202),
  (2, 0.024459084),
  (3, 0.02440217),
  (4, 0.024374727),
  (5, 0.28102407),
  (6, 0.02439529),
  (7, 0.40665495)],
 [(0, 0.013217043),
  (1, 0.045158148),
  (2, 0.46630368),
  (3, 0.13265091),
  (4, 0.013344845),
  (5,

In [93]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
BE = pyLDAvis.gensim.prepare(BE_lda, doc_term_matrix_BE, dictionary_BE)
pyLDAvis.save_html(BE, 'BE_LDA.html')

C:\Users\Java\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [25]:
#依據數值大小作排序
#Returns the indices that would sort an array.
x = np.array([3, 1, 2])
np.argsort(x)


array([1, 2, 0], dtype=int64)

In [26]:
np.array(vocab)[np.argsort(topic_dist)]

array(['church', 'pope', 'years', ..., 'gruelling', 'rapidly', 'jailed'],
      dtype='<U18')

In [30]:
doc = [["不知道", "問題", "發問", "先請"], ["演奏", "研究生", "問題","意思", "畢業", "大學生"]]

In [33]:
from gensim import corpora
# Creating term dictionary of corpus, where each unique term is assigned an index. 
dictionary = corpora.Dictionary(doc)
dictionary
doc_term_matrix = [dictionary.doc2bow(x) for x in doc]
doc_term_matrix

[[(0, 1), (1, 1), (2, 1), (3, 1)],
 [(2, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]]

In [32]:
theterms = [['jays', 'place', 'great', 'subway'],['described', 'communicative', 'friendly'], ['great', 'sarahs', 'apartament', 'back'],['great', 'sarahs', 'apartament', 'back']] 

dictionary = corpora.Dictionary(theterms)